In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl
from scipy.optimize import curve_fit

from scipy.interpolate import interp1d

from scipy.interpolate import CubicSpline, RegularGridInterpolator


In [ ]:
1+1

In [ ]:
mpl.rc('font', size=14)

dosave = True
plotqual = "data_"
savedir = "plots_11_14_23/%s" % plotqual
# DATA
filedir = "/icarus/data/users/gputnam/DMCP2023G/data/"
f = filedir + "numiRun1_unblind_reprodC_chi2u40_chi2p80_protonhit.df"

# # MC
# filedir = "/icarus/data/users/gputnam/DMCP2023G/mc/"
# f = filedir + "mcnuphase2_protonhit.df"

isMC = False

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

plt.rcParams.update({'font.size': 14})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

THXW_CORRECT = True
cut_TPCEE = False

In [ ]:
# Constants
if isMC:
    LAr_density_gmL = 1.389875
else:
    LAr_density_gmL = 1.3926
# RECOMBINATION

# ArgoNeuT params
MODA = 0.930
MODB = 0.212
Wion = 1e3 / 4.237e7

if isMC: # MC Efield
    Efield = 0.494
else: # data efield
    Efield = 0.4926

def recombination(dEdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)

    dQdx = np.log(alpha + dEdx*beta) / (Wion * beta)
    return dQdx

def recombination_cor(dQdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)

    dEdx = (np.exp(dQdx*Wion*beta)- alpha) / beta

    return dEdx


In [ ]:
# MEAN ENEGY LOSS
mass_electron = 0.5109989461 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-K-plus-minus.pdf
muon_mass = 105.6583745 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-muon.pdf

proton_mass = 938.272
Zval = 18.0
Aval = 39.948

if isMC:
    Ival = 188e-6
else:
    Ival = 197.0e-6

Ar_molar_mass = 39.9623
Ar_ZA = 18. / Ar_molar_mass
Relec = 2.817940 * 1e-13
mole = 6.0221409*1e23
Kfactor = 4*np.pi*mole*Relec**2*mass_electron # 0.307075

def Calc_MEAN_DEDX(T, thisIval=Ival, mass=proton_mass, z=1):
    gamma = (mass+T)/mass
    beta = np.power(1.0-np.power(gamma,-2.0),0.5)
    Wmax = (2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0))/(1.0+2.0*gamma*(mass_electron/mass)+np.power(mass_electron/mass,2.0))

    # Medium energy 
    dens_factor = 2.0*np.log(10)*np.log10(beta*gamma)-5.2146+0.19559*np.power(3.0-np.log10(beta*gamma),3.0)
    # low energy
    dens_factor[np.log10(beta*gamma) < 0.2] = 0.
    dens_factor[beta < 1e-6] = 0.
    # high energy
    dens_factor[np.log10(beta*gamma) > 3.0] = (2.0*np.log(10)*np.log10(beta*gamma)-5.2146)[np.log10(beta*gamma) > 3.0]
    dEdx_mean = z*LAr_density_gmL*Kfactor*(Zval/Aval)*np.power(beta,-2.0)*(0.5*np.log(2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0)*Wmax*np.power(thisIval,-2.0))-np.power(beta,2.0)-dens_factor/2.0)

    return dEdx_mean

def Calc_RR_points(KE, dRR=0.01, mass=muon_mass, z=1, thisIval=Ival, do_recombine=False):
    thisKE = KE
    KE_points = [0.]
    RR_points = [0.]

    while thisKE > 0.0:
        deltaKE = Calc_MEAN_DEDX(np.array([thisKE]), mass=mass, z=z, thisIval=thisIval) * dRR
        RR_points.append(RR_points[-1] + dRR)
        deltaKE = deltaKE[0]
        thisKE -= deltaKE
        if do_recombine:
            deltaKE = recombination(deltaKE/dRR)*dRR

        KE_points.append(KE_points[-1]+deltaKE)

    # KE_points = np.array(list(reversed(KE_points[:-1])))
    KE_points = np.array(KE_points)
    KE_points = np.flip(KE_points[-1] - KE_points)
    RR_points = np.array(RR_points)

    return KE_points, RR_points

def Calc_Q2KE_points(KE, recomb, dQ0=500, mass=muon_mass, z=1):
    thisKE = KE
    KE_points = []
    Q_points = []
    while thisKE > 0.0:
        dEdx = Calc_MEAN_DEDX(np.array([thisKE]), mass=mass, z=1)
        dQdx = recomb(dEdx)
        dx = dQ0/dQdx[0]
        deltaKE = dEdx * dx
        dQ = dQdx*dx
        Q_points.append(dQ)
        thisKE -= deltaKE[0]
        KE_points.append(thisKE)

    KE_points = np.flip(np.array(KE_points[:-1]), axis=0)
    Q_points = np.cumsum(np.flip(np.array(Q_points[:-1]), axis=0), axis=0)

    return KE_points, Q_points


In [ ]:
# Range to Energy
KE_points_max = 1000.
KE_points, RR_points = Calc_RR_points(KE_points_max, mass=muon_mass)
RR2KE_mu = CubicSpline(RR_points, KE_points)

KE_points, RR_points = Calc_RR_points(KE_points_max, mass=proton_mass)
RR2KE_p = CubicSpline(RR_points, KE_points)


In [ ]:
def dedx(dqdx, gain=1., A=MODA, B=MODB):
    return recombination_cor(dqdx*gain, A, B).rename("dedx")

def dedxdf(hitdf, gain=1, A=0.93, dqdxname="dqdx", RR_QE=5):
    df = pd.concat([dedx(hitdf[dqdxname], gain=gain, A=A, B=hitdf.beta), hitdf[dqdxname].rename("dqdx"), hitdf.rr, hitdf.pitch], axis=1)

    # get min/max RR
    minrr = df.groupby(level=list(range(df.index.nlevels-1))).rr.min().rename("minrr")
    maxrr = df.groupby(level=list(range(df.index.nlevels-1))).rr.max().rename("maxrr")

    df = df.join(minrr)
    df = df.join(maxrr)

    # Compute the charge-energy
    if RR_QE > 0:
        qsum = (hitdf[dqdxname]*hitdf.pitch*(hitdf.rr<RR_QE))[::-1].groupby(level=list(range(df.index.nlevels-1))).cumsum()
        Bs = hitdf.beta.groupby(level=list(range(df.index.nlevels-1))).first().values
        def this_recomb(dEdx):
            return recombination(dEdx, A=A, B=Bs)/gain
        KEs, Qs = Calc_Q2KE_points(float(RR2KE_p(RR_QE*50)), this_recomb, mass=proton_mass)

        i_func = 0
        def lookup_E(x):
            nonlocal i_func
            ind = np.searchsorted(Qs[:, i_func], x)
            E = KEs[ind]
            i_func += 1
            return E

        q_ke_p = qsum.groupby(level=list(range(df.index.nlevels-1))).transform(lookup_E)
        q_ke_p.name = "qke_p"
        df = df.join(q_ke_p)

    return df



In [ ]:
def emb_beta(phi, beta90=0.205, R=1.25):
    phirad = phi*np.pi/180
    return beta90 / np.sqrt(np.sin(phirad)**2 + np.cos(phirad)**2/R**2)

if isMC:
    ALPHA = 0.93
    GAIN = 78.1
    
    # same in either case
    GAIN_GAINFIT = 78.1
    
else:
    ALPHA = 0.906
    GAIN = 75.1
    
    GAIN_GAINFIT = 78.9

In [ ]:
ex_phis = np.array([66.8, 0, 90])
ex_betas = emb_beta(ex_phis)

for i,(B,P) in enumerate(zip(ex_betas, ex_phis)):
    def this_recomb(dEdx):
        return recombination(dEdx, A=ALPHA, B=B)/GAIN

    KEs, Qs = Calc_Q2KE_points(200, this_recomb, mass=proton_mass)
    
    ss = " (NuMI Beam)" if i == 0 else ""
    
    plt.plot(Qs[KEs > 10], KEs[KEs > 10], label="$\\phi: %.0f^\\circ$%s" % (P, ss))
    plt.legend(title="Proton Angle")
    
    plt.xlabel("Charge [$e^-$]")
    plt.ylabel("Kinetic Energy [MeV]")
#     plt.yscale("log")
#     plt.xscale("log")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "charge_to_energy_map.pdf")

In [ ]:
data = pd.read_hdf(f, key="phit")

In [ ]:
data = data[data.rr < 30].copy()
data

In [ ]:
# data.columns = [c[1] if isinstance(c, tuple) else c for c in data.columns]

In [ ]:
data["itpc"] = data.cryostat*2 + data.tpc // 2

In [ ]:
data["tpcEE"] = data.itpc == 0
data["tpcEW"] = data.itpc == 1
data["tpcWE"] = data.itpc == 2
data["tpcWW"] = data.itpc == 3

In [ ]:
def fid(data, iny=20, inz=50, inxcathode=15, inxanode=10):
    ymax = 134
    ymin = -180
    
    zmin = -900
    zmax = 900
    
    # per-TPC xmin, xmax
    xmin0 = -358.49
    xmax0 = -210.29
    xmin1 = -210.14
    xmax1 = -61.94
    
    xmin2 = 61.94
    xmax2 = 210.14
    xmin3 = 210.29
    xmax3 = 358.49
    
    fidX = ((data.x > xmin0 + inxanode) & (data.x < xmax0 - inxcathode)) |\
           ((data.x > xmin1 + inxcathode) & (data.x < xmax1 - inxanode)) |\
           ((data.x > xmin2 + inxanode) & (data.x < xmax2 - inxcathode)) |\
           ((data.x > xmin3 + inxcathode) & (data.x < xmax3 - inxanode))
    
    fid = (data.y > ymin + iny) & (data.y < ymax - iny)\
        & (data.z < zmax - inz) & (data.z > zmin + inz)\
        & fidX
    
    if not isMC:
        # Cut out some problem regions in the detector
        fid = fid & (np.abs(data.z) > 10)

        # TPC EW -- not actually that bad
#         bad_tpcEW = data.tpcEW & (data.z > 700) & (data.y < 0)
        bad_tpcEW = False

        # TPC WW
        bad_tpcWW = data.tpcWW & (data.y > 80) & (data.z > 0)

        fid = fid & ~bad_tpcEW & ~bad_tpcWW
        
    if cut_TPCEE: # is TPC EE borked?
        fid = fid & ~data.tpcEE
    
    return fid

In [ ]:
data["fid"] = fid(data)

In [ ]:
if ("allfid", "") in data.columns:
    del data[("allfid", "")]

data = data.join(data.fid.groupby(level=[0,1,2]).all().rename(("allfid", "")))

In [ ]:
if isMC:
    data["dqdx_nom"] = data.integral / data.pitch
else:
    data["dqdx_nom"] = data.dqdx
    
data["phi"] = np.arccos(np.abs(data.dir_x))*180/np.pi
data["thxw"] = np.abs(np.arctan(data.dir_x*data.pitch/0.3)*180/np.pi)

data["thit"] = (data.t - 850)*0.4

In [ ]:
if isMC:
    data.dqdx = data.dqdx_nom * np.exp(data.thit/3e3)

In [ ]:
# Whether to do angle correction
if THXW_CORRECT:
    thxws = np.array([5, 10, 20, 30, 40, 50, 60, 70])
    if "thxw_bin" in data.columns:
        del data["thxw_bin"]
    data["thxw_bin"] = np.searchsorted(thxws, data.thxw.values) - 1
    data = data[(data.thxw_bin >= 0 ) & (data.thxw_bin < (len(thxws)-1))].copy()
    
    if isMC:
        correction = np.array([1.        , 0.99720325, 0.99156045, 0.98546707, 0.9793954 ,
               0.97420377, 0.97335549])
    else:
        correction = np.array([1.        , 0.99610562, 0.99034266, 0.98329818, 0.97787762,
               0.97512796, 0.97599466])
        
    data.dqdx = data.dqdx / correction[data.thxw_bin]

In [ ]:
# Gain-only calibration

In [ ]:
data["beta"] = 0.212

In [ ]:
trkhitdedx_nom = dedxdf(data[data.allfid], gain=GAIN_GAINFIT, A=0.93)

In [ ]:
if isMC:
    data["beta"] = 0.212
else:
    data["beta"] = emb_beta(data.phi)

In [ ]:
trkhitdedx_ang = dedxdf(data[data.allfid], gain=GAIN, A=ALPHA)

In [ ]:
dedxdf_tosave = trkhitdedx_ang.loc[data.allfid, ["dedx", "rr"]].reset_index(drop=True)
dedxdf_tosave

In [ ]:
if dosave:
    dedxdf_tosave.to_hdf(savedir + "proton_dedx.df", key="dedx")

In [ ]:
def erange_p(hitdf):
    rangegroup = hitdf.maxrr.groupby(level=list(range(hitdf.index.nlevels-1))).first()
    return pd.Series(RR2KE_p(rangegroup), rangegroup.index)

def ecal(hitdf):
    return (hitdf.dedx*hitdf.pitch).groupby(level=list(range(hitdf.index.nlevels-1))).sum()

def ecal_p(hitdf, rrcut=2):
    Q_E = (hitdf.qke_p*(hitdf.rr < rrcut)).groupby(level=list(range(hitdf.index.nlevels-1))).max()
    R_E = (hitdf.dedx*hitdf.pitch*(hitdf.rr>=rrcut)).groupby(level=list(range(hitdf.index.nlevels-1))).sum()
    return Q_E + R_E

In [ ]:
erange_p_v = erange_p(trkhitdedx_nom)

In [ ]:
ecal_nom = ecal(trkhitdedx_nom)
ecal_p_nom = ecal_p(trkhitdedx_nom, rrcut=3)

In [ ]:
ecal_ang = ecal(trkhitdedx_ang)
ecal_p_ang = ecal_p(trkhitdedx_ang, rrcut=3)

In [ ]:
etrue = data[data.fid].truth.h_e.groupby(level=[0,1,2, 3]).sum()

In [ ]:
widebins = ((np.linspace(-1, 1, 21) - 0.05) / 2)[1:]
tightbins = ((np.linspace(-1, 1, 21) - 0.05) / 4)[1:]

In [ ]:
def gauss(X, A, mu, sigma):
    return A * np.exp(-(X-mu)**2 / (2*sigma**2))

def double_gauss(X, A1, mu1, sigma1, A2, mu2, sigma2):
    return A1 * np.exp(-(X-mu1)**2 / (2*sigma1**2)) + A2 * np.exp(-(X-mu2)**2 / (2*sigma2**2)) 

In [ ]:
def energy_comparison_plot(plt, v, bins, text="left", fit_double=True, energy1="calo", energy2="range"):
    centers = (bins[1:] + bins[:-1]) / 2.
    binerr = (bins[1:] - bins[:-1]) / 2

    XS = np.linspace(bins[0], bins[-1], 1000)
    
    N,_ = np.histogram(v, bins=bins)
    plt.errorbar(centers, N, yerr=np.sqrt(N), xerr=binerr, linestyle="none", marker=".", color="black")
    
    ymax = N.max()
    
    if fit_double:
        p0 = [
            ymax/2, 0.0, 0.75,
            ymax/2, 0.0, 0.15
        ]

        popt, perr = curve_fit(double_gauss, centers, N, p0=p0, maxfev=10_000)

        p1 = popt[:3]
        p2 = popt[-3:]
        if p1[0] < p2[0]:
            p1 = popt[-3:]
            p2 = popt[:3]

        peak_val = np.max(double_gauss(XS, *popt))
        peak_ind = np.argmax(double_gauss(XS, *popt))
        half_lo = XS[np.argmin(np.abs(peak_val/2 - double_gauss(XS[:peak_ind], *popt)))]
        half_hi = XS[peak_ind + np.argmin(np.abs(peak_val/2 - double_gauss(XS[peak_ind:], *popt)))]    
        FWHM = half_hi - half_lo

        plt.plot(XS, double_gauss(XS, *popt))
        plt.plot(XS, gauss(XS, *p1), color="gray", linestyle=":")
        plt.plot(XS, gauss(XS, *p2), color="gray", linestyle="--")        
    else:
        p0 = [
            ymax, 0.0, 0.05,
        ]

        popt, perr = curve_fit(gauss, centers, N, p0=p0, maxfev=10_000)

        p1 = popt[:3]

        peak_val = np.max(gauss(XS, *popt))
        peak_ind = np.argmax(gauss(XS, *popt))
        half_lo = XS[np.argmin(np.abs(peak_val/2 - gauss(XS[:peak_ind], *popt)))]
        half_hi = XS[peak_ind + np.argmin(np.abs(peak_val/2 - gauss(XS[peak_ind:], *popt)))]    
        FWHM = half_hi - half_lo

        plt.plot(XS, gauss(XS, *popt))
    
    trans = plt.gca().get_yaxis_transform()
    
    if text == "left":
        x_lo = 0.025
        x_hi = 0.35
        x_center = 0.1
        x_fwhm = 0.7
    else:
        x_lo = 0.6
        x_hi = 0.975
        x_center = 0.7
        x_fwhm = 0.05
    
    if fit_double:
        plt.axhline(ymax*0.8, x_lo, x_hi, color="gray", linestyle=":")    
        plt.text(x_lo, ymax*0.85, "$\\mu_1: %.1f$%%, $\\sigma_1: %.1f$%%" % (p1[1]*100, np.abs(p1[2])*100),
            transform=trans, fontsize=12)
        
        plt.axhline(ymax*0.6, x_lo, x_hi, color="gray", linestyle="--")    
        
        plt.text(x_lo, ymax*0.65, "$\\mu_2: %.1f$%%, $\\sigma_2: %.1f$%%" % (p2[1]*100, np.abs(p2[2])*100),
                transform=trans, fontsize=12)
    
        plt.text(x_center, ymax*0.5, "$N_1/N_2: %.2f$" % (p1[0] / p2[0]),
                transform=trans, fontsize=12)
    else:
        plt.text(x_lo, ymax*0.85, "$\\mu: %.1f$%%, $\\sigma: %.1f$%%" % (p1[1]*100, np.abs(p1[2])*100),
            transform=trans, fontsize=12)
        
    plt.text(x_fwhm, 0.9, "FWHM: %.0f%%" % (FWHM*100), fontsize=12, transform=plt.gca().transAxes)
    
    if fit_double:
        plt.text(x_lo+0.005, 0.875, "Double Gaussian Fit", fontsize=12, transform=plt.gca().transAxes)
    else:
        plt.text(x_lo+0.005, 0.875, "Single Gaussian Fit", fontsize=12, transform=plt.gca().transAxes)
        
    if isMC:
        plt.text(x_fwhm, 0.6, "ICARUS\nMC", fontsize=20, transform=plt.gca().transAxes)
    else:
        plt.text(x_fwhm, 0.6, "ICARUS\nData", fontsize=20, transform=plt.gca().transAxes)
    
    plt.xlabel("$(E_\\mathrm{%s} - E_\\mathrm{%s}) / E_\\mathrm{%s}$" % (energy1, energy2, energy2))
    plt.ylabel("Proton-Like Tracks")

In [ ]:
if isMC:
    bins = tightbins
else:
    bins = widebins

In [ ]:
ediff = (ecal_nom - erange_p_v) / erange_p_v

energy_comparison_plot(plt, ediff, widebins)
plt.title("ArgoNeuT Recombination")

if dosave: plt.savefig(savedir + "proton_Ereco_nominalrecomb.pdf", bbox_inches="tight")

In [ ]:
ediff = (ecal_p_nom - erange_p_v) / erange_p_v

energy_comparison_plot(plt, ediff, widebins)
plt.title("ArgoNeuT Recombination + Q-Tip")

if dosave: plt.savefig(savedir + "proton_Ereco_nominalrecomb_qtip.pdf", bbox_inches="tight")

In [ ]:
ediff = (ecal_ang - erange_p_v) / erange_p_v

energy_comparison_plot(plt, ediff, widebins)
plt.title("EMB Recombination")

if dosave: plt.savefig(savedir + "proton_Ereco_angularrecomb.pdf", bbox_inches="tight")

In [ ]:
ediff = (ecal_p_ang - erange_p_v) / erange_p_v

energy_comparison_plot(plt, ediff, widebins)
plt.title("EMB Recombination + Q-Tip")

if dosave: plt.savefig(savedir + "proton_Ereco_angularrecomb_qtip.pdf", bbox_inches="tight")

In [ ]:
ediff = (erange_p_v - etrue) / etrue

energy_comparison_plot(plt, ediff, tightbins, energy1="range", energy2="true")

if dosave: plt.savefig(savedir + "proton_Erange_Etrue.pdf", bbox_inches="tight")

In [ ]:
ediff = (ecal_p_ang - etrue) / etrue

energy_comparison_plot(plt, ediff, tightbins, energy1="calo", energy2="true")

if dosave: plt.savefig(savedir + "proton_Ecalo_Etrue.pdf", bbox_inches="tight")